In [ ]:
import SimpleITK as sitk
import skimage.io as io
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mping
import os
from PIL import Image
import tensorflow as tf
import keras
from keras import backend as k
from keras.datasets import mnist
from keras.models import Sequential,Model
from keras.layers import Conv2D,MaxPooling2D,Dropout,Flatten,Dense,Input,UpSampling2D,concatenate,core
from skimage import morphology
from skimage import measure
from sklearn.cluster import KMeans
from skimage.transform import resize
import random

In [ ]:
def get_subdirectory(path):
    dirList = []
    files = os.listdir(path)
    for f in files:
        if(os.path.isdir(path + '/' + f)):
            if(f[0] == '.'):
                pass #排除隐藏文件夹
            else:
                dirList.append(path+ '/' + f)
    return dirList

def get_ap(path):
    img_pathList = []
    name = 'arterial phase'
    subpath = get_subdirectory(path)
    pathnumber = len(subpath)
    for i in range(pathnumber):
        img_pathList.append(subpath[i] + '/' + name)
    return img_pathList

def get_image(path):
    return [os.path.join(path,f) for f in os.listdir(path) if f.endswith('.png')]

#读取当前文件夹下的所有DICOM文件，并以数组形式存储在image_array中，
def Xyread(path):
    reader = sitk.ImageSeriesReader()
    dicom_names = reader.GetGDCMSeriesFileNames(path)
    reader.SetFileNames(dicom_names)
    image = reader.Execute()
    image_array = sitk.GetArrayFromImage(image)
    X = image_array
    mean = np.mean(X)
    std = np.std(X)
    X = X-mean
    X = X/std
    Xshape = X.shape
    
    c = get_image(path)
    d = len(c)
    
    mask = np.empty((d,512,512))
    i = 0
    while(d>0):
        img = Image.open(c[d-1])
        img_ndarray = np.asarray(img)/255
        mask[i] = img_ndarray
        d = d-1
        i = i+1
    return X,mask

path = r"./dataset"
path_total= get_ap(path)


In [ ]:
def improcess(X,y,patchsize,patchnum):
    s = patchsize
    index1 = 200
    index2 = 450
    index3 = 150
    index4 = 400
    len = X.shape[0]*patchnum
    X_train = np.zeros([len,s,s])
    y_train = np.zeros([len,s,s])
    ncol = X[0].shape[1]
    for i in range(0,X.shape[0]):
        X_slice = X[i]
        y_slice = y[i]
        #choosing 
        indexrow = np.random.randint( int(index1+s/2)+1 , int(index2-s/2)-1, size = patchnum ) 
        indexcol = np.random.randint( int(index3+s/2)+1 , int(index4-s/2)-1 , size = patchnum )
        for j in range(0,patchnum):
            X_train[j+i*patchnum,:,:]=X_slice[int(indexrow[j]-s/2):int(indexrow[j]+s/2),int(indexcol[j]-s/2):int(indexcol[j]+s/2)]
            y_train[j+i*patchnum,:,:] = y_slice[int(indexrow[j]-s/2):int(indexrow[j]+s/2),int(indexcol[j]-s/2):int(indexcol[j]+s/2)]
    
    return index1,index2,X_train,y_train

In [ ]:
def get_unet(n_ch,patch_height,patch_width):
    inputs = Input(shape=(patch_height,patch_width,n_ch))
    conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(inputs)
    conv1 = Dropout(0.2)(conv1)
    conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv1)
    pool1 = MaxPooling2D((2, 2))(conv1)
    #
    conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(pool1)
    conv2 = Dropout(0.2)(conv2)
    conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv2)
    pool2 = MaxPooling2D((2, 2))(conv2)
    #
    conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(pool2)
    conv3 = Dropout(0.2)(conv3)
    conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv3)

    up1 = UpSampling2D(size=(2, 2))(conv3)
    up1 = concatenate([conv2,up1],axis=3)
    conv4 = Conv2D(64, (3, 3), activation='relu', padding='same')(up1)
    conv4 = Dropout(0.2)(conv4)
    conv4 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv4)
    #
    up2 = UpSampling2D(size=(2, 2))(conv4)
    up2 = concatenate([conv1,up2], axis=3)
    conv5 = Conv2D(32, (3, 3), activation='relu', padding='same')(up2)
    conv5 = Dropout(0.2)(conv5)
    conv5 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv5)
    #  注意，写成这种结构，并且用的loss为categorical_crossentropy，
    # 需要对groundtruth数据进行处理，见后面help_function.py里的mask_Unet
    conv6 = Conv2D(2, (1, 1), activation='relu',padding='same')(conv5)
    conv6 = core.Reshape((2,patch_height*patch_width))(conv6)
    conv6 = core.Permute((2,1))(conv6)
    ############
    conv7 = core.Activation('softmax')(conv6)

    model = Model(inputs=inputs, outputs=conv7)

    # sgd = SGD(lr=0.01, decay=1e-6, momentum=0.3, nesterov=False)
    model.compile(optimizer='sgd', loss='categorical_crossentropy',metrics=['accuracy'])

    return model

In [ ]:
def masks_Unet(masks):
    assert (len(masks.shape)==4)  #4D arrays
    assert (masks.shape[3]==1 )  #check the channel is 1
    im_h = masks.shape[1]
    im_w = masks.shape[2]
    masks = np.reshape(masks,(masks.shape[0],im_h*im_w))
    new_masks = np.empty((masks.shape[0],im_h*im_w,2))
    for i in range(masks.shape[0]):
        for j in range(im_h*im_w):
            if  masks[i,j] == 0:
                new_masks[i,j,0]=1
                new_masks[i,j,1]=0
            else:
                new_masks[i,j,0]=0
                new_masks[i,j,1]=1
    return new_masks

In [ ]:
X,mask = Xyread(path_total[0])
for i in range(1,100):
    X_temp,mask_temp = Xyread(path_total[i])
    X = np.append(X,X_temp,axis = 0)
    mask = np.append(mask,mask_temp,axis = 0)
    
X_predict_array,mask = Xyread(path_total[100])
for j in range(101,107):
    X_temp1,mask_temp1 = Xyread(path_total[j])
    X_predict_array = np.append(X_predict_array,X_temp1,axis=0)

In [ ]:
X = np.load("C:/Users/cassi/Desktop/CT_withtumor_input.npy")
mask = np.load("C:/Users/cassi/Desktop/mask_withtumor_input.npy")

In [ ]:
seed = 1
index1,index2,X_train,y_train = improcess(X,mask,48,100)
img_rows = X_train.shape[1]
img_cols = X_train.shape[2]
X_train = X_train.reshape(X_train.shape[0], img_rows,img_cols,1)
y_train = y_train.reshape(y_train.shape[0],img_rows,img_cols,1)
#y_train = masks_Unet(y_train)

In [ ]:
np.unique(np.where(y_train == 1)[0]).shape

In [ ]:
y_train.shape[0]

In [ ]:
y_train_1 = masks_Unet(y_train)

In [ ]:
np.unique(np.where(y_train_1 == 1)[0])

In [ ]:
y_train_1 = y_train_1[:,:,1].reshape(y_train.shape[0],img_rows,img_cols,1)

In [ ]:
plt.figure()
plt.subplot(1,2,1)
plt.imshow(X_train[330,:,:,0])
plt.subplot(1,2,2)
plt.imshow(y_train_1[330,:,:,0])
plt.show()

In [ ]:
n_ch = X_train.shape[3]#number of chanels
patch_height = X_train.shape[1]
patch_width = X_train.shape[2]
model = get_unet(n_ch, patch_height, patch_width)  #the U-net model
print("Check: final output of the network:")
print(model.output_shape)

In [ ]:
N_epochs = 36
batch_size = 64

model.fit(X_train, y_train, epochs=N_epochs, batch_size=batch_size, verbose=1, shuffle=True, validation_split=0.1)

In [ ]:
n_test = X_predict_array.shape[0]
X_predict = np.zeros([100*n_test,48,48,1])
for k in range(n_test):
    index = 0
    for i in range(10):
        for j in range(10):
            X_predict[k+index,:,:,0] = X_predict_array[k,32+i*48:80+i*48,32+j*48:80+j*48]
            index = index + 1
            
mask_predict = model.predict(X_predict,verbose = 1)
mask_show = mask_predict[:,:,0]
mask_show = mask_show.reshape(n_test,480,480)


In [ ]:
model.predict(X_predict[4].reshape(1,48,48,1),verbose = 1)

In [ ]:
mask_show = np.zeros((18200,2304))
mask_show[predict_index] = 1
mask_show = mask_show.reshape(n_test,480,480)

In [ ]:
np.where(mask_show == 1)

In [ ]:
model.save("C:/Users/cassi/Desktop/Unet_maskonly.h5")

In [ ]:
X_predict_array = np.load("C:/Users/cassi/Desktop/CT_test.npy")
y_predict_array = np.load("C:/Users/cassi/Desktop/mask_test.npy")

In [ ]:
plt.imshow(mask_show[2],'gray')
plt.show()